In [ ]:
# -*- coding: utf-8 -*-

from PIL import Image, ImageDraw, ImageFont
import barcode
from barcode.writer import ImageWriter
import io

# --- 配置项 ---

# 定义城市和颜色的映射关系
CITY_COLOR_MAP = {
    'cs': (255, 0, 0),      # 红色
    'zz': (0, 255, 0),      # 绿色
    'xt': (0, 0, 255),      # 蓝色
    'hy': (255, 255, 0),    # 黄色
    'sy': (255, 165, 0),   # 橙色
    'yuey': (128, 0, 128),    # 紫色
    'zjj': (0, 0, 0),        # 黑色
    'yiy': (255, 255, 255),  # 白色
    'cd': (0, 255, 255),    # 青色
    'ld': (255, 192, 203),  # 粉色
    'cz': (128, 128, 128),  # 灰色
    'yz': (0, 128, 0),      # 深绿色
    'hh': (0, 0, 128),      # 海军蓝
}

# 尝试加载一个中文字体，确保能显示中文
# 在Windows上通常可以在 C:/Windows/Fonts/ 找到
# 在macOS上通常在 /System/Library/Fonts/ 或 /Library/Fonts/
# 在Linux上通常在 /usr/share/fonts/
try:
    # 尝试使用常见的黑体
    font_path = "simhei.ttf" 
    main_font = ImageFont.truetype(font_path, 40)
    small_font = ImageFont.truetype(font_path, 30)
except IOError:
    print(f"无法找到字体文件 '{font_path}'。将使用默认字体，中文可能无法显示。")
    main_font = ImageFont.load_default()
    small_font = ImageFont.load_default()


def create_logistics_label(cargo_id: str, city: str, file_path: str):
    """
    创建一个易于识别的物流单图片。

    :param cargo_id: 货物ID (例如: 'BJ-2025-0912-001')
    :param city: 目的地城市 (例如: '北京')
    :param file_path: 生成图片保存的路径 (例如: 'label.png')
    """
    if city not in CITY_COLOR_MAP:
        raise ValueError(f"城市 '{city}' 不在预定义的颜色映射中。")

    # 1. 创建画布
    label_width, label_height = 800, 800
    bg_color = (255, 255, 255)
    image = Image.new('RGB', (label_width, label_height), bg_color)
    draw = ImageDraw.Draw(image)

    # 2. 绘制特征点 (左上角) - 黑方块内嵌白方块再内嵌黑方块
    margin = 20
    # 外层黑方块
    draw.rectangle([(margin, margin), (margin + 100, margin + 100)], fill=(0, 0, 0))
    # 中层白方块
    draw.rectangle([(margin + 20, margin + 20), (margin + 80, margin + 80)], fill=(255, 255, 255))
    # 内层黑方块
    draw.rectangle([(margin + 40, margin + 40), (margin + 60, margin + 60)], fill=(0, 0, 0))


    # 3. 绘制颜色块
    color = CITY_COLOR_MAP[city]
    color_block_start_x = margin + 100 + 10
    draw.rectangle(
        [(color_block_start_x, margin), (color_block_start_x + 450, margin + 100)],
        fill=color
    )
    # 4. 绘制文本信息
    text_color = (0, 0, 0)
    draw.text((margin, margin + 150), f"CITY：{city}", fill=text_color, font=small_font)
    # 货物ID
    draw.text((margin, margin + 200), f"ID：{cargo_id}", fill=text_color, font=small_font)
    
    # 5. 生成并粘贴条形码
    # 使用 Code 128 格式
    code128 = barcode.get_barcode_class('code128')
    
    # 将条形码渲染到内存中的BytesIO对象，而不是临时文件
    barcode_io = io.BytesIO()
    # writer_options 可以控制条码的各种属性
    writer_options = {
        "module_height": 20.0,  # 条码高度
        "font_size": 15,        # 条码下方文字大小
        "text_distance": 6.0,   # 条码与文字的距离
        "quiet_zone": 5,        # 两边留白
    }
    barcode_img = code128(cargo_id, writer=ImageWriter()).write(barcode_io, writer_options)
    
    # 从内存中读取条形码图片
    barcode_io.seek(0)
    barcode_pil_img = Image.open(barcode_io)
    
    # 调整条形码尺寸并粘贴到画布上
    # barcode_pil_img = barcode_pil_img.resize((label_width - margin * 4, 100)) # 可以强制缩放，但可能影响识别
    paste_x = margin + 10
    paste_y = label_height - barcode_pil_img.height - margin-100
    image.paste(barcode_pil_img, (paste_x, paste_y))

    # 6. 保存最终的物流单图片
    image.save(file_path)
    print(f"物流单 '{file_path}' 已成功生成！")


# --- 调用示例 ---
if __name__ == '__main__':
    create_logistics_label(
        cargo_id='CS-2025-0912-001', 
        city='cs', 
        file_path='label1.png'
    )

    create_logistics_label(
        cargo_id='ZZ-2025-0912-002', 
        city='zz', 
        file_path='label2.png'
    )
    create_logistics_label(
        cargo_id='XT-2025-0912-003', 
        city='xt', 
        file_path='label3.png'
    )

物流单 'label1.png' 已成功生成！
物流单 'label2.png' 已成功生成！
物流单 'label3.png' 已成功生成！


In [ ]:
# -*- coding: utf-8 -*-
"""
ArUco 角标版物流单生成（图片在二维码上方、角标四角、小字体文本在二维码下方）
依赖：
  pip install pillow qrcode[pil] opencv-python opencv-contrib-python numpy

变更点：
- 图片移动到二维码上方（与二维码同列居中），两者不重叠。
- 四个 ArUco 角标放在四个角，尺寸更小；内容区自动避开角标。
- 文本移动到二维码下方，使用更小字号。
- 保留 detect_label() 示例：识别四个角标并估计整图单应矩阵。
"""
from typing import Tuple, Optional, Dict, List
from PIL import Image, ImageDraw, ImageFont
import qrcode
import numpy as np
import cv2

# -------- 配置项 --------
CITY_COLOR_MAP: Dict[str, Tuple[int, int, int]] = {
    'ChangSha': (255, 0, 0),
    'ZhuZhou': (0, 255, 0),
    'XiangTan': (0, 0, 255),
    'HengYang': (255, 255, 0),
    'ShaoYang': (255, 165, 0),
    'YueYang': (128, 0, 128),
    'ZhangJiaJie': (0, 0, 0),
    'YiYang': (255, 255, 255),
    'ChangDe': (0, 255, 255),
    'LouDi': (255, 192, 203),
    'ChenZhou': (128, 128, 128),
    'YongZhou': (0, 128, 0),
    'HuaiHua': (0, 0, 128),
    'XiangXi': (255, 20, 147),
}

# 字体（尽量用中文黑体，找不到则回退默认字体）
try:
    font_path = "simhei.ttf"
    TINY_FONT = ImageFont.truetype(font_path, 18)  # 很小
except Exception:
    TINY_FONT = ImageFont.load_default()

# -------- ArUco 工具函数 --------

def _get_aruco_dictionary(dict_name: str = "DICT_4X4_50"):
    if not hasattr(cv2, 'aruco'):
        raise RuntimeError("缺少 cv2.aruco 模块，请安装 opencv-contrib-python")
    aruco = cv2.aruco
    if hasattr(aruco, 'getPredefinedDictionary'):
        return aruco.getPredefinedDictionary(getattr(aruco, dict_name))
    return getattr(aruco, dict_name)


def make_aruco_marker(id_: int, side: int = 100, border_bits: int = 2,
                      dict_name: str = "DICT_4X4_50") -> Image.Image:
    """生成带静区的 ArUco 方形标记（返回 PIL.Image）。
    side: 最终方形边长（像素），包含白色静区。
    border_bits: 白色静区边宽（像素）。
    """
    dic = _get_aruco_dictionary(dict_name)
    inner = side - 2 * border_bits
    if inner <= 4:
        raise ValueError("side 太小或 border_bits 太大，导致内部面积不足")

    try:
        marker = cv2.aruco.generateImageMarker(dic, id_, inner)
    except Exception:
        marker = np.zeros((inner, inner), dtype=np.uint8)
        cv2.aruco.drawMarker(dic, id_, inner, marker, 1)

    canvas = np.full((side, side), 255, dtype=np.uint8)
    canvas[border_bits:border_bits+inner, border_bits:border_bits+inner] = marker
    pil_img = Image.fromarray(canvas, mode='L').convert('RGB')
    return pil_img


def paste_center(image: Image.Image, patch: Image.Image, center_xy: Tuple[int, int]):
    cx, cy = center_xy
    w, h = patch.size
    image.paste(patch, (int(cx - w/2), int(cy - h/2)))

# -------- 主函数：生成物流单 --------

def _text_block_height(draw: ImageDraw.ImageDraw, lines: List[str], font: ImageFont.ImageFont, spacing: int = 4) -> int:
    h = 0
    for i, line in enumerate(lines):
        bbox = draw.textbbox((0, 0), line, font=font)
        line_h = bbox[3] - bbox[1]
        h += line_h
        if i < len(lines) - 1:
            h += spacing
    return h


def create_logistics_label(
    cargo_id: str,
    city: str,
    file_path: str,
    top_right_img_path: Optional[str] = None,  # 复用此参数名，但图片将放在二维码上方
    label_size: Tuple[int, int] = (800, 800),
    marker_side: int = 100,
    marker_border: int = 2,
    margin: int = 20,
    gap: int = 12,  # 元素垂直间距
):
    if city not in CITY_COLOR_MAP:
        raise ValueError(f"城市 '{city}' 不在预定义的颜色映射中。")

    W, H = label_size
    bg_color = (255, 255, 255)
    image = Image.new('RGB', (W, H), bg_color)
    draw = ImageDraw.Draw(image)

    # 1) 四个 ArUco 角标（0 左上、1 右上、2 右下、3 左下），尺寸小一些
    m_lu = make_aruco_marker(0, side=marker_side, border_bits=marker_border)
    m_ru = make_aruco_marker(1, side=marker_side, border_bits=marker_border)
    m_rd = make_aruco_marker(2, side=marker_side, border_bits=marker_border)
    m_ld = make_aruco_marker(3, side=marker_side, border_bits=marker_border)

    safety = 10
    half = marker_side // 2
    lu = (margin + safety + half, margin + safety + half)
    ru = (W - margin - safety - half, margin + safety + half)
    rd = (W - margin - safety - half, H - margin - safety - half)
    ld = (margin + safety + half, H - margin - safety - half)

    # 先不贴，等内容渲染完，最后贴到最上层

    # 2) 内容区：自动避开角标
    x0 = margin + safety + marker_side
    y0 = margin + safety + marker_side
    x1 = W - margin - safety - marker_side
    y1 = H - margin - safety - marker_side

    # 3) 文本（很小）在最下方，先计算高度
    lines = [f"CITY：{city}", f"ID：{cargo_id}"]
    txt_h = _text_block_height(draw, lines, TINY_FONT, spacing=4)

    # 4) 图片在二维码上方；拼装时：图片 -> 间距 -> 二维码 -> 间距 -> 文本
    # 先决定二维码尺寸，使得三者能装下
    avail_w = x1 - x0
    avail_h = y1 - y0

    # 预设图片目标高度为内容区的 0.32，最小 120
    img_target_h = max(120, int(avail_h * 0.32)) if top_right_img_path else 0
    # 文本与两个间距占用
    reserved_h = (gap if top_right_img_path else 0) + txt_h + gap
    qr_max_h = avail_h - img_target_h - reserved_h
    if qr_max_h < 140:  # 太挤则缩小图片高度留空间给二维码
        shrink = 140 - qr_max_h
        img_target_h = max(0, img_target_h - shrink)
        qr_max_h = avail_h - img_target_h - reserved_h
    qr_side = int(min(avail_w, qr_max_h))
    qr_side = max(160, qr_side)  # 保底尺寸

    # 垂直排版起点
    cursor_y = y0
    center_x = (x0 + x1) // 2

    # 4.1 图片（可选）
    if top_right_img_path:
        try:
            img = Image.open(top_right_img_path).convert("RGBA")
            # 高质量缩放：宽受限于 avail_w，高受限于 img_target_h
            try:
                resample = Image.Resampling.LANCZOS
            except AttributeError:
                resample = Image.LANCZOS
            target_w = avail_w*1.5
            target_h = img_target_h*1.5
            img.thumbnail((target_w, target_h), resample)

            # 居中粘贴
            paste_x = int(center_x - img.width / 2)
            paste_y = int(cursor_y)
            base_rgba = image.convert("RGBA")
            overlay = Image.new("RGBA", base_rgba.size, (0, 0, 0, 0))
            overlay.paste(img, (paste_x, paste_y), img)
            image = Image.alpha_composite(base_rgba, overlay).convert("RGB")
            draw = ImageDraw.Draw(image)  # 关键：重新获取 draw，避免后续绘制到旧图像
            cursor_y = paste_y + img.height + gap
        except Exception as e:
            print(f"插入顶部图片失败: {e}")
            # 若失败，忽略图片，直接进入二维码

    # 4.2 二维码（白底遮挡）
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(cargo_id)
    qr.make(fit=True)
    qr_img = qr.make_image(fill_color="black", back_color="white").convert('RGB')
    qr_img = qr_img.resize((qr_side, qr_side))
    qr_x = int(center_x - qr_side / 2)
    qr_y = int(cursor_y)

    # 白色背景边框
    draw.rectangle([(qr_x - 10, qr_y - 10), (qr_x + qr_side + 10, qr_y + qr_side + 10)], fill=(255, 255, 255))
    image.paste(qr_img, (qr_x, qr_y))
    cursor_y = qr_y + qr_side + gap

    # 4.3 文本（很小）置于二维码下方，居中
    draw = ImageDraw.Draw(image)  # 保险：在绘制文本前再次确保 draw 指向最新的 image
    text_color = (0, 0, 0)
    for i, line in enumerate(lines):
        bbox = draw.textbbox((0, 0), line, font=TINY_FONT)
        line_w = bbox[2] - bbox[0]
        line_h = bbox[3] - bbox[1]
        draw.text((int(center_x - line_w/2), int(cursor_y)), line, fill=text_color, font=TINY_FONT)
        cursor_y += line_h + (4 if i < len(lines)-1 else 0)

    # 5) 角标最后贴到最上层，保证不被覆盖
    paste_center(image, m_lu, lu)
    paste_center(image, m_ru, ru)
    paste_center(image, m_rd, rd)
    paste_center(image, m_ld, ld)

    # 6) 保存
    image.save(file_path)
    print(f"物流单 '{file_path}' 已成功生成！")

# -------- 识别侧（可选）：检测角标并估计整图单应性 --------

def detect_label(image_path: str, expected_size: Tuple[int, int] = (800, 800)):
    """检测图中的 ArUco 0,1,2,3 角标，返回估计的单应矩阵和透视展开图。
    - 仅作示例：用角标中心点做 4 对应点来估计 H；实际工程可改用每个角标外侧最近的角点以提高精度。
    """
    img_bgr = cv2.imread(image_path)
    if img_bgr is None:
        raise FileNotFoundError(image_path)
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    dic = _get_aruco_dictionary("DICT_4X4_50")
    params = cv2.aruco.DetectorParameters()

    if hasattr(cv2.aruco, 'ArucoDetector'):
        detector = cv2.aruco.ArucoDetector(dic, params)
        corners, ids, _ = detector.detectMarkers(gray)
    else:
        corners, ids, _ = cv2.aruco.detectMarkers(gray, dic, parameters=params)

    if ids is None or len(ids) == 0:
        raise RuntimeError("未检测到任何 ArUco 标记")

    ids = ids.flatten()
    id2center: Dict[int, Tuple[float, float]] = {}
    for i, idv in enumerate(ids):
        pts = corners[i].reshape(-1, 2)
        cx, cy = float(pts[:, 0].mean()), float(pts[:, 1].mean())
        id2center[int(idv)] = (cx, cy)

    required = [0, 1, 2, 3]
    if not all(k in id2center for k in required):
        raise RuntimeError(f"未检全四角标，检测到: {sorted(id2center.keys())}")

    src = np.array([id2center[0], id2center[1], id2center[2], id2center[3]], dtype=np.float32)
    W, H = expected_size
    dst = np.array([[0, 0], [W-1, 0], [W-1, H-1], [0, H-1]], dtype=np.float32)

    H_mat, inliers = cv2.findHomography(src, dst, method=cv2.RANSAC, ransacReprojThreshold=2.0)
    if H_mat is None:
        raise RuntimeError("findHomography 失败")

    warped = cv2.warpPerspective(img_bgr, H_mat, (W, H))
    return H_mat, warped




if __name__ == '__main__':
    create_logistics_label(
        cargo_id='ChangSha-2025-0912-001',
        city='ChangSha',
        file_path='label1.png',
        top_right_img_path='C:\\Users\\29082\\Desktop\\GX\\pic\\2025-09-15 105132.png' 
    )
    create_logistics_label(
        cargo_id='ZhuZhou-2025-0912-002',
        city='ZhuZhou',
        file_path='label2.png',
        top_right_img_path='C:\\Users\\29082\\Desktop\\GX\\pic\\屏幕截图 2025-09-15 110344.png' 
    )
    create_logistics_label(
        cargo_id='XiangTan-2025-0912-003',
        city='XiangTan',
        file_path='label3.png',
        top_right_img_path='C:\\Users\\29082\\Desktop\\GX\\pic\\屏幕截图 2025-09-15 110727.png' 
    )
    create_logistics_label(
        cargo_id='HengYang-2025-0914-002',
        city='HengYang',
        file_path='label4.png',
        top_right_img_path='C:\\Users\\29082\\Desktop\\GX\\pic\\屏幕截图 2025-09-15 110803.png' 
    )
    create_logistics_label(
        cargo_id='ShaoYang-2025-0914-005',
        city='ShaoYang',
        file_path='label5.png',
        top_right_img_path='C:\\Users\\29082\\Desktop\\GX\\pic\\屏幕截图 2025-09-15 110834.png' 
    )
    create_logistics_label(
        cargo_id='ChangSha-2025-0912-006',
        city='ChangSha',
        file_path='label6.png',
        top_right_img_path='C:\\Users\\29082\\Desktop\\GX\\pic\\屏幕截图 2025-09-15 110901.png' 
    )
    create_logistics_label(
        cargo_id='HengYang-2025-0918-006',
        city='HengYang',
        file_path='label7.png',
        top_right_img_path='C:\\Users\\29082\\Desktop\\GX\\pic\\屏幕截图 2025-09-15 110936.png' 
    )
    create_logistics_label(
        cargo_id='ChangSha-2025-0908-006',
        city='ChangSha',
        file_path='label8.png',
        top_right_img_path='C:\\Users\\29082\\Desktop\\GX\\pic\\屏幕截图 2025-09-15 111117.png' 
    )
    create_logistics_label(
        cargo_id='ChangSha-2025-0902-009',
        city='ChangSha',
        file_path='label9.png',
        top_right_img_path='C:\\Users\\29082\\Desktop\\GX\\pic\\屏幕截图 2025-09-15 111139.png' 
    )
    create_logistics_label(
        cargo_id='XiangTan-2025-0916-001',
        city='XiangTan',
        file_path='label10.png',
        top_right_img_path='C:\\Users\\29082\\Desktop\\GX\\pic\\屏幕截图 2025-09-15 111151.png' 
    )
    # 可选：演示识别（将某张生成的图跑一次）
    # try:
    #     Hm, warped = detect_label('label1.png')
    #     print('估计到的单应矩阵 H:\n', Hm)
    #     cv2.imwrite('label1_warped.png', warped)
    #     print("已保存透视展开图 label1_warped.png")
    # except Exception as e:
    #     print("识别示例失败：", e)

物流单 'label1.png' 已成功生成！
物流单 'label2.png' 已成功生成！
物流单 'label3.png' 已成功生成！
物流单 'label4.png' 已成功生成！
物流单 'label5.png' 已成功生成！
物流单 'label6.png' 已成功生成！
